In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import scipy.io as sio
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import hdc

%matplotlib notebook

In [9]:
# location of all offline data
dataDir = './emg_mat/offline/'

baseExperiment = 1
newExperiment = 3

holdStart = 70
holdEnd = 149
D = 10000

numIter = 1

numSVM = np.zeros((5,numIter))
accSVM = np.zeros((5,numIter))
accHDC = np.zeros((5,numIter))

# for s in range(5):
for s in [1]:
    # subject labels are 1-indexed
    subject = s + 1
    print('Running Subject ' + str(subject))
    
    # load data from the two contexts
    filename = dataDir + 'S' + str(subject) + 'E' + str(baseExperiment) + '.mat'
    base = sio.loadmat(filename)['emgHD']
    filename = dataDir + 'S' + str(subject) + 'E' + str(newExperiment) + '.mat'
    new = sio.loadmat(filename)['emgHD']
    
    # get metatdata
    numGest, numTrial = base.shape
    numCh = base[0,0][2].shape[1]
    numEx = holdEnd - holdStart + 1
    
    # iterate through to get averages (cross-validation)
    for n in range(numIter):
        
        # separate training and testing dataframes
        features = np.empty((numCh*5,0))
        ngrams = np.empty((D,0))
        labels = np.empty(0)
        trials = np.empty(0)
        isTest = np.empty(0)
        context = np.empty(0)
        
        # collect baseline data
        testTrials = np.random.randint(0,numTrial,numGest)
        for g in range(numGest):
            for t in range(numTrial):
                trial = base[g,t]
                feat = np.empty((0,numEx))
                for i in range(5):
                    feat = np.concatenate((feat,trial[2][(holdStart+i):(holdEnd+i+1),:].T),axis=0)
                features = np.concatenate((features,feat),axis=1)
                ngrams = np.concatenate((ngrams,trial[3][:,holdStart:holdEnd+1]),axis=1)
                labels = np.concatenate((labels,g*np.ones(numEx)))
                trials = np.concatenate((trials,t*np.ones(numEx)))
                if t == testTrials[g]:
                    isTest = np.concatenate((isTest,np.ones(numEx)))
                else:
                    isTest = np.concatenate((isTest,np.zeros(numEx)))
                context = np.concatenate((context,0*np.ones(numEx)))
        # collect new data
        testTrials = np.random.randint(0,numTrial,numGest)
        for g in range(numGest):
            for t in range(numTrial):
                trial = new[g,t]
                feat = np.empty((0,numEx))
                for i in range(5):
                    feat = np.concatenate((feat,trial[2][(holdStart+i):(holdEnd+i+1),:].T),axis=0)
                features = np.concatenate((features,feat),axis=1)
                ngrams = np.concatenate((ngrams,trial[3][:,holdStart:holdEnd+1]),axis=1)
                labels = np.concatenate((labels,g*np.ones(numEx)))
                trials = np.concatenate((trials,t*np.ones(numEx)))
                if t == testTrials[g]:
                    isTest = np.concatenate((isTest,np.ones(numEx)))
                else:
                    isTest = np.concatenate((isTest,np.zeros(numEx)))
                context = np.concatenate((context,1*np.ones(numEx)))

        # create dataframe for features
        featCols = ['feature' + str(i) for i in range(features.shape[0])]
        featData = pd.DataFrame(features.T,columns=featCols)
        featData['gesture'] = labels
        featData['trial'] = trials
        featData['isTest'] = isTest
        featData['context'] = context
        
        # create dataframe for ngrams
        ngramCols = ['hv' + str(i) for i in range(ngrams.shape[0])]
        ngramData = pd.DataFrame(ngrams.T,columns=ngramCols)
        ngramData['gesture'] = labels
        ngramData['trial'] = trials
        ngramData['isTest'] = isTest
        ngramData['context'] = context
        
        # train HD model
        allGest = ngramData['gesture'].unique()
        AM = hdc.Memory(D)
        for g in allGest:
            ng = np.asarray(ngramData.loc[(ngramData['gesture'] == g) & (ngramData['isTest'] == 0) & (ngramData['context'] == 0)].iloc[:,0:D])
            for i in range(ng.shape[0]):
                AM.train(ng[i,:],name=str(int(g)) + 'A')

        for g in allGest:
            ng = np.asarray(ngramData.loc[(ngramData['gesture'] == g) & (ngramData['isTest'] == 0) & (ngramData['context'] == 1)].iloc[:,0:D])
            for i in range(ng.shape[0]):
                AM.train(ng[i,:],name=str(int(g)) + 'B')
            
        # collect testing data and perform inference
        testNgram = ngramData.loc[(ngramData['isTest'] == 1)].iloc[:,0:D]
        testLabel = ngramData.loc[(ngramData['isTest'] == 1)].iloc[:,D]
        
        AM.bipolarize()
        sim,label = AM.find(np.asarray(testNgram))
        label = np.asarray([int(l[:-1]) for l in label])
        accHDC[s,n] = (label == np.asarray(testLabel)).sum()/len(label)
        acc = accHDC[s,np.nonzero(accHDC[s,:])].mean()
        print('\t HDC Iteration %d: Accuracy = %f'%(n+1,acc))
        
        # train and test SVM model
        numFeat = 320
        clf = svm.SVC(decision_function_shape='ovo',kernel='linear',C=10)
        clf.fit(featData.loc[featData['isTest'] == 0].iloc[:,0:numFeat],featData.loc[featData['isTest'] == 0].iloc[:,numFeat])
        yhat = clf.predict(featData.loc[featData['isTest'] == 1].iloc[:,0:numFeat])
        accSVM[s,n] = accuracy_score(yhat,featData.loc[featData['isTest'] == 1].iloc[:,numFeat])
        numSVM[s,n] = len(clf.support_)
        acc = accSVM[s,np.nonzero(accSVM[s,:])].mean()
        num = numSVM[s,np.nonzero(numSVM[s,:])].mean()
        print('\t Iteration %d: SVM Accuracy = %f'%(n+1,acc))
        
# #         # batch train and test LDA model
# #         clf = LinearDiscriminantAnalysis()
# #         clf.fit(Xtrain,ytrain)
# #         yhat = clf.predict(Xtest)
# #         print('\t Iteration %d: Accuracy = %f\r'%(n+1,accuracy_score(yhat,ytest)),end="")
# # #         accSVM[s,n] = accuracy_score(yhat,ytest)
# # #         numSVM[s,n] = len(clf.support_)
# #         print('')

Running Subject 2
	 HDC Iteration 1: Accuracy = 0.987019
	 Iteration 1: SVM Accuracy = 0.993269
